In [14]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import time
from sqlalchemy import inspect, create_engine
import matplotlib.pyplot as plt
import numpy as np

In [15]:
#Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# HOSTNAME = "localhost"
# PORT = 5432
# DATABASE = "diapers_db"
# USERNAME = "postgres"
# PASSWORD = "password"
# path = f'postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}:{PORT}/{DATABASE}'
# engine = create_engine(path)

In [ ]:
# conn = engine.connect()

In [16]:
#Three websites searching Data Analyst positions
dice = "https://www.dice.com/jobs?q=data%20analyst&countryCode=US&radius=30&radiusUnit=mi&page=1&pageSize=20&language=en&eid=S2Q_,bw_0"
#indeed_lyst = "https://www.indeed.com/jobs?q=data+analyst&l=&from=searchOnHP&vjk=f43749e32d279328"
#glass_lyst = "https://www.glassdoor.com/Job/data-analyst-jobs-SRCH_KO0,12.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=data%2520analyst"


browser.visit(dice)



In [17]:
html = browser.html
html[0:500]

'<html lang="en" xmlns:ng="http://angularjs.org" class=""><head><style type="text/css">@charset "UTF-8";[ng\\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide{display:none !important;}ng\\:form{display:block;}</style>\n    \n\n\n\n    <style>.async-hide { opacity: 0 !important} </style>\n<script src="//bat.bing.com/bat.js" async=""></script><script src="https://connect.facebook.net/signals/config/412179360593168?v=2.9.90&amp;r=stable" async=""></script><script async="" src="h'

In [18]:
soup = BeautifulSoup(browser.html, "html.parser")
print(soup.prettify())

<html class="" lang="en" xmlns:ng="http://angularjs.org">
 <head>
  <style type="text/css">
   @charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide{display:none !important;}ng\:form{display:block;}
  </style>
  <style>
   .async-hide { opacity: 0 !important}
  </style>
  <script async="" src="//bat.bing.com/bat.js">
  </script>
  <script async="" src="https://connect.facebook.net/signals/config/412179360593168?v=2.9.90&amp;r=stable">
  </script>
  <script async="" src="https://connect.facebook.net/en_US/fbevents.js">
  </script>
  <script async="" src="https://www.redditstatic.com/ads/pixel.js">
  </script>
  <script async="" src="https://snap.licdn.com/li.lms-analytics/insight.min.js" type="text/javascript">
  </script>
  <script async="" src="https://bat.bing.com/bat.js" type="text/javascript">
  </script>
  <script async="" src="https://www.google-analytics.com/gtm/optimize-dyn.js?id=GTM-WL6BH3V&amp;cid=184892509.1671385114&amp;cb=0.663

In [19]:
soup= BeautifulSoup(browser.html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [23]:
#Find code to loop through web page and pull the title of the job and possibly the location, then find code to click the job and acquire the salary.
browser.find_by_text('Data')


AttributeError: 'WebDriver' object has no attribute 'link'

In [ ]:
#Set-up sleep function
# browser.visit(dice_lyst)
# time.sleep(2)